In [2]:
!git clone https://github.com/BESSAI-Tayeb/genie-project.git
%cd genie-project

Cloning into 'genie-project'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 109 (delta 0), reused 0 (delta 0), pack-reused 107 (from 2)
Receiving objects: 100% (109/109), 107.24 MiB | 12.25 MiB/s, done.
Resolving deltas: 100% (23/23), done.
/content/genie-project


In [3]:
!pip install nerfstudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.9 MB/s eta 0:00:00
  Using cached pybind11-3.0.1-py3-none-any.whl.metadata (10.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 580.8/580.8 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 110.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 

In [8]:
# Create the target directory first
%cd genie/knn

/content/genie-project/genie/knn


In [25]:
# create folder for optix installation
!mkdir -p ./NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64

In [ ]:
# Try running the installer with the directory already created
!bash ../../scripts/NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64-35015278.sh --skip-license --prefix=./NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64

# # If that still fails, try extracting manually (the .sh is actually a tar.gz inside)
# !tar -xvf ../../scripts/NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64-35015278.sh --strip-components=1 -C ./NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64

# Check what was extracted
!ls -la

NVIDIA OptiX Installer Version: 8.1.0, Copyright (c) NVIDIA Corporation
This is a self-extracting archive.
The archive will be extracted to: ./NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64

Using target directory: ./NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64
Extracting, please wait...

Unpacking finished successfully
tar: This does not look like a tar archive
tar: Skipping to next header
tar: Exiting with failure status due to previous errors
total 72
drwxr-xr-x 3 root root  4096 Nov 30 09:48 .
drwxr-xr-x 7 root root  4096 Nov 30 09:37 ..
-rw-r--r-- 1 root root  3541 Nov 30 09:37 bindings.cpp
-rwxr-xr-x 1 root root  3168 Nov 30 09:37 build_optix.sh
-rw-r--r-- 1 root root  1649 Nov 30 09:37 Header.cuh
-rw-r--r-- 1 root root  8275 Nov 30 09:37 knn_algorithms.py
-rw-r--r-- 1 root root 27495 Nov 30 09:37 KNN.cu
drwxr-xr-x 5 root root  4096 Nov 30 09:49 NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64
-rw-r--r-- 1 root root  6537 Nov 30 09:37 shaders.cu


In [27]:
# Check what was extracted
!ls -la NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64/

# Look for the SDK directory structure
!find NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64/ -name "include" -type d

# Check for optix.h header
!find NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64/ -name "optix.h"

total 384
drwxr-xr-x  5 root root   4096 Nov 30 09:49 .
drwxr-xr-x  3 root root   4096 Nov 30 09:48 ..
drwxr-xr-x  2 root root   4096 Oct 21  2024 doc
drwxr-xr-x  3 root root   4096 Oct 21  2024 include
-rw-r--r--  1 root root 355775 Oct 21  2024 OptiX_EndUserLicense.pdf
-rw-r--r--  1 root root  14011 Oct 21  2024 OptiX_ThirdParty_Licenses.txt
drwxr-xr-x 37 root root   4096 Oct 21  2024 SDK
NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64/SDK/support/GLFW/include
NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64/SDK/optixMixSDKs/OptiX SDK 7.5.0/include
NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64/include
NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64/SDK/optixMixSDKs/OptiX SDK 7.5.0/include/optix.h
NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64/include/optix.h


In [29]:
!ls NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64/include/

internal			   optix_function_table.h  optix_stack_size.h
optix_denoiser_tiling.h		   optix.h		   optix_stubs.h
optix_device.h			   optix_host.h		   optix_types.h
optix_function_table_definition.h  optix_micromap.h


In [32]:
!ls -la

total 72
drwxr-xr-x 3 root root  4096 Nov 30 09:48 .
drwxr-xr-x 7 root root  4096 Nov 30 09:37 ..
-rw-r--r-- 1 root root  3541 Nov 30 09:37 bindings.cpp
-rwxr-xr-x 1 root root  3168 Nov 30 09:37 build_optix.sh
-rw-r--r-- 1 root root  1649 Nov 30 09:37 Header.cuh
-rw-r--r-- 1 root root  8275 Nov 30 09:37 knn_algorithms.py
-rw-r--r-- 1 root root 27495 Nov 30 09:37 KNN.cu
drwxr-xr-x 5 root root  4096 Nov 30 09:49 NVIDIA-OptiX-SDK-8.1.0-linux64-x86_64
-rw-r--r-- 1 root root  6537 Nov 30 09:37 shaders.cu


In [33]:
# Install python3-dev package for Colab
!apt-get update -qq
!apt-get install -y python3-dev

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Reading package lists... Done%
Building dependency tree... Done
Reading state information... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  javascript-common libjs-sphinxdoc libjs-underscore libpython3.10
  libpython3.10-dev libpython3.10-minimal libpython3.10-stdlib python3.10
  python3.10-dev python3.10-minimal
Suggested packages:
  apache2 | lighttpd | httpd python3.10-venv python3.10-doc binfmt-support
The following NEW packages will be installed:
  javascript-common libjs-sphinxdoc libjs-underscore python3-dev
  python3.10-dev
The following packages will be upgraded:
  libpython3.10 libpython3.10-dev libpython3.10-minimal libpython3.10-stdlib
  python3.10 python3.10-minimal
6 upgraded, 5

In [34]:
# Now build the OptiX KNN module using the existing build script
# We should already be in genie/knn/ directory

!bash ./build_optix.sh

# Check if the module was built successfully
!ls -lh optix_knn.so

🔍 Detecting GPU Compute Capability...
Detected CUDA_ARCH=sm_75
🔍 Detecting CUDA include path...
Detected CUDA include path: /usr/local/cuda/include
🔍 Detecting Thrust include path...
Detected Thrust include path: /usr/local/cuda/include
Found libcuda.so at /usr/local/cuda-12.5/targets/x86_64-linux/lib/stubs
Found libcuda.so at /usr/local/cuda-12.5/targets/x86_64-linux/lib/stubs
🔍 Detecting PyTorch ABI flag...
🔍 Detecting PyTorch ABI flag...
Detected ABI: CXX11 ABI enabled
📦 Compiling shaders.cu to PTX...
Detected ABI: CXX11 ABI enabled
📦 Compiling shaders.cu to PTX...
shaders.cu(9): warning #20044-D: extern declaration of the entity optixLaunchParams is treated as a static definition

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

🔧 Compiling KNN.cu...
nvcc fatal   : Incompatible code generation requested: arch='compute_86', code='sm_75'
shaders.cu(9): warning #20044-D: extern declaration of the entity optixLaunchParams is treated as a static definition


In [ ]:
# Verify the OptiX KNN module works
!python3 -c "import sys; sys.path.insert(0, '.'); import optix_knn; print('✅ OptiX KNN module loaded successfully!')"

-rwxr-xr-x 1 root root 2.8M Nov 29 10:37 optix_knn.so


In [ ]:
# Navigate back to the genie-project root directory
%cd /content/genie-project

# Install GENIE in editable mode
!pip install -e .

# Install Nerfstudio CLI
!ns-install-cli

✅ OptiX KNN module loaded successfully!


In [ ]:
# Test if GENIE commands are available
!ns-train --help | head -20

# Check if genie-export and genie-render are available
!which genie-export
!which genie-render

# Verify the OptiX KNN module is accessible from the installed package
!python3 -c "import sys; sys.path.insert(0, 'genie/knn'); import optix_knn; print('✅ OptiX KNN module works!')"

/kaggle/working/genie
Obtaining file:///kaggle/working/genie
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for nerfstudio-genie (pyproject.toml) ... done
  Created wheel for nerfstudio-genie: filename=nerfstudio_genie-0.1.0-0.editable-py3-none-any.whl size=4110 sha256=1d9a90d13f9d0d087e15993c95e05c33554c8d55510cf0683a09524d434abaa5
  Stored in directory: /tmp/pip-ephem-wheel-cache-5or5pqc3/wheels/55/c7/11/fe4a9b621982ad9dfad1f4f074e0e5fc2e260ec3e9d6522a83
Successfully built nerfstudio-genie
[10:37:57] 🤷 .zshrc not found, skipping.                                                                 ]8;id=677364;file:///usr/local/lib/python3.11/dist-packages/nerfstudio/scripts/completions/install.py\install.py]8;;\:]8;id=765160;file:///usr/local/lib/python3.11/dist-packages/nerfstudio/scripts/completions/ins